---
**License**

 stats_dist_rosenbrock

 Mon Jan 25 20:56:00 2020\
 Copyright  2021\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 stats_dist_rosenbrock\
 Copyright (C) 2021 Sandro Dias Pinto Vitenti <vitenti@uel.br>


 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
import sys
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatterMathtext
import numpy as np

from scipy.stats import binned_statistic
import matplotlib.ticker as mtick

from IPython.display import HTML, display
import tabulate

from numcosmo_py import Ncm

from numcosmo_py.plotting.tools import confidence_ellipse
from numcosmo_py.plotting.tools import plot_m2lnp
from numcosmo_py.plotting.tools import set_rc_params_article

%matplotlib inline

In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
dim = 2
nps = 160
nps_test = 100000

split_fraction = 0.6

rng = Ncm.RNG.seeded_new(None, 123)

sigma1 = math.sqrt(10.0)
sigma2 = 1.0 / sigma1
mu1 = 1.0


def m2lnL_val(va):
    return (mu1 - va[0]) ** 2 / sigma1**2 + (va[1] - va[0] ** 2) ** 2 / sigma2**2

In [ ]:
interps = []
interps_desc = []

kernel0 = Ncm.StatsDistKernelST.new(dim, 1.0)
interp0 = Ncm.StatsDistKDE.new(kernel0, Ncm.StatsDistCV.NONE)
interps.append(interp0)
interps_desc.append("Interp-KDE:Cauchy")

kernel1 = Ncm.StatsDistKernelST.new(dim, 3.0)
interp1 = Ncm.StatsDistKDE.new(kernel1, Ncm.StatsDistCV.NONE)
interps.append(interp1)
interps_desc.append("Interp-KDE:ST3")

kernel2 = Ncm.StatsDistKernelGauss.new(dim)
interp2 = Ncm.StatsDistKDE.new(kernel2, Ncm.StatsDistCV.NONE)
interps.append(interp2)
interps_desc.append("Interp-KDE:Gauss")

kernel3 = Ncm.StatsDistKernelST.new(dim, 1.0)
interp3 = Ncm.StatsDistVKDE.new(kernel3, Ncm.StatsDistCV.NONE)
interps.append(interp3)
interps_desc.append("Interp-VKDE:Cauchy")

kernel4 = Ncm.StatsDistKernelST.new(dim, 3.0)
interp4 = Ncm.StatsDistVKDE.new(kernel4, Ncm.StatsDistCV.NONE)
interps.append(interp4)
interps_desc.append("Interp-VKDE:ST3")

kernel5 = Ncm.StatsDistKernelGauss.new(dim)
interp5 = Ncm.StatsDistVKDE.new(kernel5, Ncm.StatsDistCV.NONE)
interps.append(interp5)
interps_desc.append("Interp-VKDE:Gauss")

In [ ]:
for interp in interps:
    interp.reset()

theta_train = []  # Training set variables
m2lnp_train = []  # Training set m2lnp

for i in range(nps):
    x1 = rng.gaussian_gen(mu1, sigma1)
    x2 = rng.gaussian_gen(x1 * x1, sigma2)

    theta_i = [x1, x2]
    m2lnp_i = m2lnL_val(theta_i)

    theta_train.append(theta_i)
    m2lnp_train.append(m2lnp_i)

    theta_v_i = Ncm.Vector.new_array(theta_i)
    for interp in interps:
        interp.add_obs(theta_v_i)

theta_train = np.array(theta_train)
m2lnp_train = np.array(m2lnp_train)
m2lnp_train_v = Ncm.Vector.new_array(m2lnp_train)

theta_test = []  # Test set variables
m2lnp_test = []  # Test set m2lnp

for i in range(nps_test):
    x1 = rng.gaussian_gen(mu1, sigma1)
    x2 = rng.gaussian_gen(x1 * x1, sigma2)

    theta_i = [x1, x2]
    m2lnp_i = m2lnL_val(theta_i)

    theta_test.append(theta_i)
    m2lnp_test.append(m2lnp_i)

    theta_v_i = Ncm.Vector.new_array(theta_i)

theta_test = np.array(theta_test)
m2lnp_test = np.array(m2lnp_test)

In [ ]:
ml = max([len(interp_desc) for interp_desc in interps_desc])

for interp, interp_desc in zip(interps, interps_desc):
    interp.set_cv_type(Ncm.StatsDistCV.SPLIT)
    interp.set_split_frac(split_fraction)
    interp.set_print_fit(False)

    interp.prepare_interp(m2lnp_train_v)
    calib_os = interp.get_over_smooth()

    print(
        f"Calibrate {interp_desc:<{ml}} interpolation "
        f"with os = {calib_os:15.8g} and rnorm = {interp.get_rnorm():15.8g}"
    )

In [ ]:
m2lnp_interps = []

for interp in interps:
    m2lnp_interp = []
    for theta in theta_test:
        m2lnp_interp.append(interp.eval_m2lnp(Ncm.Vector.new_array(theta)))
    m2lnp_interp = np.array(m2lnp_interp)

    m2lnp_interps.append(m2lnp_interp)

In [ ]:
set_rc_params_article(ncol=2, nrows=2)

plotn = 250
vmin = 1.0e-8

gs = mpl.gridspec.GridSpec(3, 3, wspace=0.0, hspace=0.4, height_ratios=[1, 1, 2])
fig = plt.figure()
axa = []

axa.append(fig.add_subplot(gs[0, 0]))
axa.append(fig.add_subplot(gs[0, 1], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[0, 2], sharex=axa[0], sharey=axa[0]))

axa.append(fig.add_subplot(gs[1, 0], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[1, 1], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[1, 2], sharex=axa[0], sharey=axa[0]))
axT = plt.subplot(gs[2, :], sharex=axa[0], sharey=axa[0])

x1_a = np.linspace(-9, 9, plotn)
x2_a = np.linspace(-4, 80, plotn)

for interp, interp_desc, ax0 in zip(interps, interps_desc, axa):
    z = np.array(
        [
            interp.eval_m2lnp(Ncm.Vector.new_array([x1, x2]))
            for x2 in x2_a
            for x1 in x1_a
        ]
    )

    plot_m2lnp(x1_a, x2_a, z, ax0, plotn=plotn, vmin=vmin)

    for i in range(0, interp.get_n_kernels()):
        y_i, cov_i, n_i, w_i = interp.get_Ki(i)
        mu = y_i.dup_array()
        cov = np.array([[cov_i.get(i, j) for j in range(2)] for i in range(2)])
        confidence_ellipse(mu, cov, ax0, n_std=4, edgecolor="red", lw=0.1)

    ax0.set_title(interp_desc)
    ax0.set_xlabel("$x_1$")

for i, ax in enumerate(axa):
    if i == 0 or i == 3:
        ax.set_ylabel("$x_2$")
    else:
        plt.setp(ax.get_yticklabels(), visible=False)

zT = np.array([m2lnL_val([x1, x2]) for x2 in x2_a for x1 in x1_a])
img = plot_m2lnp(x1_a, x2_a, zT, axT, plotn=plotn, vmin=vmin)

axT.set_title("True Rosenbrock")
axT.set_xlabel("$x_1$")
axT.set_ylabel("$x_2$")

fig.colorbar(img, format=LogFormatterMathtext())
plt.savefig("rosenbrock_kernel_interp.pdf", bbox_inches="tight")

pass

In [ ]:
set_rc_params_article(ncol=2, nrows=1)

plotn = 250
vmin = 1.0e-8

fig = plt.figure()
ax = plt.gca()

x1_a = np.linspace(-9, 9, plotn)
x2_a = np.linspace(-4, 80, plotn)

zT = np.array([m2lnL_val([x1, x2]) for x2 in x2_a for x1 in x1_a])
img = plot_m2lnp(x1_a, x2_a, zT, ax, plotn=plotn, vmin=vmin)

ax.set_title("True Rosenbrock")
ax.set_xlabel("$x_1$")
ax.set_ylabel("$x_2$")

fig.colorbar(img, format=LogFormatterMathtext())
plt.savefig("rosenbrock_exact.pdf", bbox_inches="tight")

pass

In [ ]:
set_rc_params_article(ncol=2, nrows=1)

plotn = 250
vmin = 1.0e-8

gs = mpl.gridspec.GridSpec(2, 3, wspace=0.0, hspace=0.4, height_ratios=[1, 1])
fig = plt.figure()
axa = []

axa.append(fig.add_subplot(gs[0, 0]))
axa.append(fig.add_subplot(gs[0, 1], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[0, 2], sharex=axa[0], sharey=axa[0]))

axa.append(fig.add_subplot(gs[1, 0], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[1, 1], sharex=axa[0], sharey=axa[0]))
axa.append(fig.add_subplot(gs[1, 2], sharex=axa[0], sharey=axa[0]))

x1_a = np.linspace(-9, 9, plotn)
x2_a = np.linspace(-4, 80, plotn)

for interp, interp_desc, ax0 in zip(interps, interps_desc, axa):
    z = np.array(
        [
            interp.eval_m2lnp(Ncm.Vector.new_array([x1, x2]))
            for x2 in x2_a
            for x1 in x1_a
        ]
    )

    plot_m2lnp(x1_a, x2_a, z, ax0, plotn=plotn, vmin=vmin)

    for i in range(0, interp.get_n_kernels()):
        y_i, cov_i, n_i, w_i = interp.get_Ki(i)
        mu = y_i.dup_array()
        cov = np.array([[cov_i.get(i, j) for j in range(2)] for i in range(2)])
        confidence_ellipse(mu, cov, ax0, n_std=4, edgecolor="red", lw=0.1)

    ax0.set_title(interp_desc)
    ax0.set_xlabel("$x_1$")

for i, ax in enumerate(axa):
    if i == 0 or i == 3:
        ax.set_ylabel("$x_2$")
    else:
        plt.setp(ax.get_yticklabels(), visible=False)

fig.colorbar(img, format=LogFormatterMathtext())
plt.savefig("rosenbrock_kernel_interp_only.pdf", bbox_inches="tight")

pass

In [ ]:
table = [
    [
        "Interpolation",
        r"|\tilde{\pi}/\pi-1| < 20%",
        r"mean $\alpha$",
        "Split Fraction",
        "Over-Smooth",
    ]
]

log_diff_array = []
log_prob_array = []

for interp, interp_desc, m2lnp_interp in zip(interps, interps_desc, m2lnp_interps):
    p_test = np.exp(-0.5 * m2lnp_test)
    p_interp = np.exp(-0.5 * m2lnp_interp)

    log_diff = -0.5 * (m2lnp_interp - m2lnp_test)
    rel_diff = np.abs(np.expm1(log_diff))

    log_prob = -0.5 * (
        (m2lnp_interp[0::2] - m2lnp_test[0::2])
        - (m2lnp_interp[1::2] - m2lnp_test[1::2])
    )
    log_prob = np.clip(log_prob, np.log(1.0e-14), 0.0)
    prob = np.exp(log_prob)

    log_diff_array.append(np.clip(log_diff, np.log(1.0e-3), np.log(1.0e3)))
    log_prob_array.append(log_prob)

    qp = [0.05, 0.5]

    q_rel_diff = np.quantile(rel_diff, qp) * 100.0
    q_prob = np.quantile(prob, qp) * 100.0

    line = [interp_desc]
    line.append(f"{sum(rel_diff<0.3)*100.0/len(rel_diff):.0f}%")
    line.append(f"{100.0*np.mean(prob):.0f}%")
    line.append(f"{split_fraction * 100.0:.0f}%")
    line.append(f"{interp.get_over_smooth():.2f}")
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt="html")))

In [ ]:
set_rc_params_article(ncol=1, nrows=2)

gs = mpl.gridspec.GridSpec(2, 1, hspace=0.0)
fig = plt.figure()
axa = []

axa.append(fig.add_subplot(gs[0, 0]))
axa.append(fig.add_subplot(gs[1, 0], sharex=axa[0]))

p_test_2 = p_test[::2]
index_array_prob = np.argsort(p_test_2)
x_prob = np.log(p_test_2[index_array_prob])

index_array_rel_diff = np.argsort(p_test)
x_rel_diff = np.log(p_test[index_array_rel_diff])

lstyles = [
    ("k", "-", "o"),
    ("k", "--", "v"),
    ("k", ":", "x"),
    ("b", "-", "o"),
    ("b", "--", "v"),
    ("b", ":", "x"),
]
bins = np.logspace(-4, 0, 7)

for desc, log_diff, log_prob, lstyle in zip(
    interps_desc, log_diff_array, log_prob_array, lstyles
):
    s, edges, _ = binned_statistic(
        np.exp(x_rel_diff),
        np.abs(np.expm1(log_diff[index_array_rel_diff])),
        statistic=lambda x: 100.0 * sum(x < 0.20) / len(x),
        bins=bins,
    )
    axa[0].hlines(
        s,
        edges[:-1],
        edges[1:],
        color=lstyle[0],
        linestyle=lstyle[1],
        label=desc,
        lw=0.5,
    )

    s, edges, _ = binned_statistic(
        np.exp(x_prob),
        np.exp(log_prob[index_array_prob]),
        statistic=lambda x: 100.0 * sum(x) / len(x),
        bins=bins,
    )
    axa[1].hlines(
        s,
        edges[:-1],
        edges[1:],
        color=lstyle[0],
        linestyle=lstyle[1],
        label=desc,
        lw=0.5,
    )

plt.setp(axa[0].get_xticklabels(), visible=False)
axa[0].yaxis.set_major_formatter(mtick.PercentFormatter())
axa[1].yaxis.set_major_formatter(mtick.PercentFormatter())

axa[0].set_xscale("log")
axa[1].set_xscale("log")

axa[0].legend(loc="upper left")
axa[0].set_ylabel("$|\\tilde\\pi/\\pi-1| < 0.2$")
axa[1].set_ylabel("mean $\\alpha$")
axa[1].set_xlabel("$\\pi$")

plt.savefig("rosenbrock_kernel_err_acpr.pdf", bbox_inches="tight")

pass